We will be making a multimodal rag system based on gemini 2.0 (gemini-2.5-flash-preview-05-20 in my case) using this tutorial: https://medium.com/@christopher.henkel.ai/building-a-powerful-and-scalable-multimodal-rag-system-with-gemini-2-0-flash-88086536f237

First, load the necessary libraries:

In [6]:
%pip install -q google-generativeai pdf2image chromadb sentence_transformers fitz PyMuPDF

Note: you may need to restart the kernel to use updated packages.


In [7]:
from google import genai
from google.genai import types
# from sentence_transformers import SentenceTransformer
import requests
from PIL import Image
import pymupdf
import chromadb
import io, re, os
from dotenv import load_dotenv
from IPython.display import display, Markdown
import time

load_dotenv()
key = os.getenv("GEMINI_API_KEY")

# print(key)
client = genai.Client(api_key=key)
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works in a few words. Let's think step by step."
)
print(response.text)

Okay, let's break down how AI works in simple steps:

1.  **Data:** AI learns from data (lots of it!).
2.  **Patterns:** It finds patterns and relationships in that data.
3.  **Models:** It creates a model based on those patterns.
4.  **Prediction/Action:**  It uses the model to predict outcomes or take actions on new data.
5.  **Improvement:** It gets feedback and adjusts the model to improve its performance over time.

So, in a few words: **AI learns patterns from data to predict or act.**



Load the pdf document from an url and parse each page of the pdf and save it as an image.

In [8]:
pdf_url = "https://arxiv.org/pdf/2501.12948"

# download the PDF
response = requests.get(pdf_url)
print (response)
# open the PDF
pdf_document = pymupdf.open(stream=response.content, filetype="pdf")
print(pdf_document)
# save every pdf page as an image in a list
page_images = []
for page_number in range(pdf_document.page_count):
    page = pdf_document[page_number]
    print(page)
    pix = page.get_pixmap()

    # Convert PyMuPDF pixmap into PIL Image
    img_data = pix.tobytes("png")
    img = Image.open(io.BytesIO(img_data))
    page_images.append(img)

pdf_document.close()



<Response [200]>
Document('None', <memory, doc# 2>)
page 0 of <memory, doc# 2>
page 1 of <memory, doc# 2>
page 2 of <memory, doc# 2>
page 3 of <memory, doc# 2>
page 4 of <memory, doc# 2>
page 5 of <memory, doc# 2>
page 6 of <memory, doc# 2>
page 7 of <memory, doc# 2>
page 8 of <memory, doc# 2>
page 9 of <memory, doc# 2>
page 10 of <memory, doc# 2>
page 11 of <memory, doc# 2>
page 12 of <memory, doc# 2>
page 13 of <memory, doc# 2>
page 14 of <memory, doc# 2>
page 15 of <memory, doc# 2>
page 16 of <memory, doc# 2>
page 17 of <memory, doc# 2>
page 18 of <memory, doc# 2>
page 19 of <memory, doc# 2>
page 20 of <memory, doc# 2>
page 21 of <memory, doc# 2>


Visualize each page image to confirm that the parsing process actually worked.

In [9]:
# for idx, img in enumerate(page_images):
#     print(f"Displaying page {idx + 1}")
#     img.show()

Displaying page 1
Displaying page 2
Displaying page 3
Displaying page 4
Displaying page 5
Displaying page 6
Displaying page 7
Displaying page 8
Displaying page 9
Displaying page 10
Displaying page 11
Displaying page 12
Displaying page 13
Displaying page 14
Displaying page 15
Displaying page 16
Displaying page 17
Displaying page 18
Displaying page 19
Displaying page 20
Displaying page 21
Displaying page 22


OCR with Gemini 2.0 Flash (gemini-2.5-flash-preview-05-20 in my case)

Next, prompt the multimodal LLM to do the OCR with chunks. The LLM should create meaningful chunks defined by the tags < chunk>< /chunk> (without spaces). Tables should be formatted in html and additionally summarized (this will improve retrieval by adding more support). Same for figures.

In [10]:
# Maybe improve this prompt
OCR_PROMPT = """\
OCR the following document into Markdown. 
Chunk the document into sections of roughly 250 - 1000 words. 
The chunks should be meaningful and complete with no breaking of sentences.
Surround each chunk with <chunk> and </chunk> tags. 
If there are tables, format them as HTML. 
Additionally, include a summary of the findings for each table with numbers.
If there are figures or graphs, summarize the key insights using numerical data. 
Before the summary, indicate whether it is a graph and include its graph number. 
Ensure that the summary appears in the same section as the corresponding table or figure.
"""

OCR_PROMPT_2 = """\
You are an expert document processor. Your task is to OCR a document and convert it into Markdown format. The process involves the following steps:

1. Chunking: Divide the OCRed text into meaningful sections, each ranging from 250 to 1000 words. Ensure each chunk is complete and doesn't break sentences. Enclose each chunk within <chunk> and </chunk> tags.

2. Table Formatting: Format all tables as HTML. After each table, provide a summary of the table's key findings, emphasizing numerical data.

3. Figure/Graph Summarization: For each figure or graph, summarize the key insights using numerical data. Before each summary, indicate the graph type and its number (e.g., "Graph 1:"). The summary should appear in the same section as the corresponding figure/graph.

4. Output: The final output should be a Markdown document with the described chunking, table formatting, and figure/graph summarization.
"""

# Added chunking 128-512 tokens
OCR_PROMPT_3 = """
You are an expert document processor designed to extract information from images of document pages and convert it into a structured Markdown format suitable for a Retrieval-Augmented Generation (RAG) system.
Your task involves processing an image of a document page using OCR, chunking the resulting text, formatting tables, and summarizing figures and graphs. Follow these steps precisely:

1. OCR and Initial Cleanup: Perform OCR on the input image to extract the text content of the document page. Correct any obvious OCR errors (e.g., character substitutions, misspellings) where possible, but do not spend excessive time on this. The goal is to ensure the text is reasonably accurate.

2. Chunking: Divide the OCRed text into meaningful sections. Each chunk should contain between 128 and 512 tokens (350 words). The chunks should be contextually relevant and maintain sentence integrity; do not break sentences mid-way. If a sentence needs to be split across chunks, ensure the overlap between chunks includes enough preceding text to maintain context. If a paragraph exceeds 512 tokens, split it at a semantically appropriate point, ensuring no information is lost and context is preserved within each chunk. Enclose each chunk within <chunk> and </chunk> tags. Ensure all Markdown formatting is contained within the chunk.

3. Table Formatting and Summarization: Identify tables within the OCRed text. Format each table as an HTML table. Immediately following each table, provide a concise summary (2-3 sentences) of the table's key findings, emphasizing numerical data, trends, and significant values. If possible, include units. Example: "Table 1 shows the performance metrics of different models. Model A achieved the highest accuracy at 95%, while Model B had the fastest inference speed at 120ms."

4. Figure/Graph Summarization: Identify figures and graphs within the OCRed text. For each figure or graph, provide a concise summary (2-3 sentences) of the key insights derived from the visual, emphasizing numerical data, trends, and significant data points. Label each summary with the graph type and number (e.g., "Graph 1: Accuracy vs. Training Time"). Example: "Graph 2: shows that increasing the number of layers improves model performance up to 10 layers, but after that performance plateaus." The summary should appear immediately after the figure or graph description.

5. Target Audience Considerations: While the primary audience is professionals in the generative AI space, be mindful that some users may have less familiarity. Explain concepts clearly and avoid overly technical jargon where possible in your summaries.

6. Markdown Consistency: Use standard Markdown formatting where applicable. No specific Markdown flavor is required beyond basic formatting.

7. Error Handling: If the OCR process introduces errors that prevent accurate table or figure analysis, make a best effort to interpret the data, but prioritize accurate information when possible. Note any ambiguities or potential inaccuracies briefly within the corresponding summary.

8. Output: Return the entire output as a single string containing the Markdown-formatted document, including the <chunk> tags, HTML tables, and summaries, for use within a RAG system.
"""

def ocr_page(page_num, image):
    try:
        response = client.models.generate_content(
            model = "gemini-2.5-flash-preview-05-20",
            contents = [OCR_PROMPT_3, image]
        )
        page_in_ocr_format = response.text

        # parse <chunk> blocks
        chunks = re.findall(r"<chunk>(.*?)</chunk>", page_in_ocr_format, re.DOTALL)

        if not chunks:
            # if no chunk tags where created
            chunks = page_in_ocr_format.split("\n\n")
        
        chunk_blocks = []
        for idx, chunk_text in enumerate(chunks):
            # store Id, chunk text
            chunk_blocks.append({
                "id": f"page_{page_num}_chunk_{idx}",
                "text": chunk_text.strip()
            })
        print(f"Successfully processed page {page_num}")  
        return chunk_blocks
    except Exception as e:
        print(f"Error processing page {page_num}: {e}")
        return []

Don't run the below. Not all pages don't get processed. \\/ \\/

In [11]:
# # ocr and chunks each pdf page
# all_chunks = []
# for i, image in enumerate(page_images):
#     page_chunks = ocr_page(i, image)
#     all_chunks.extend(page_chunks)

# print(f"Total chunks: {len(all_chunks)}")

#### The above ocr task was not successful. Due to model overload, not all the pages were processed. (I am using a free account!!) This is being used to display the chunks gained from ocr using markdown. This helps us visualize what information the ocr truly captured beyond html tags.
#### For now, we will use what is processed for the rest of the rag system welp.

#### Had to restart the python kernel so now this is commented out!!


In [12]:
# for j in all_chunks:
#     display(Markdown(j[id]))
#     display(Markdown(j[text]))
#     # print("Text: " + chunk.text)


## Let's try this again, but with a wait so that gemini won't cry model overload.

In [13]:
import time

# ocr and chunks each pdf page with OCR_PROMPT_3 and time.sleep(1 minute)
all_chunks_again = []
for i, image in enumerate(page_images):
    page_chunks = ocr_page(i, image)
    all_chunks_again.extend(page_chunks)
    time.sleep(20)

print(f"Total chunks: {len(all_chunks_again)}")

for j in all_chunks_again:
    display(Markdown(j[id]))
    display(Markdown(j[text]))

Successfully processed page 0
Successfully processed page 1
Successfully processed page 2
Successfully processed page 3
Successfully processed page 4
Successfully processed page 5
Successfully processed page 6
Successfully processed page 7
Successfully processed page 8
Successfully processed page 9
Successfully processed page 10
Successfully processed page 11
Successfully processed page 12
Successfully processed page 13
Successfully processed page 14
Successfully processed page 15
Successfully processed page 16
Successfully processed page 17
Successfully processed page 18
Successfully processed page 19
Successfully processed page 20
Successfully processed page 21
Total chunks: 90


KeyError: <built-in function id>

In [14]:
for j in all_chunks_again:
    # print (j)
    display(Markdown(j['id']))
    display(Markdown(j['text']))

page_0_chunk_0

# DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning

**DeepSeek-AI**
research@deepseek.com

## Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1. DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without supervised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities. Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing reasoning behaviors. However, it encounters challenges such as poor readability and language mixing. To address these issues and further enhance reasoning performance, we introduce DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models (1.5B, 7B, 8B, 14B, 32B, 70B) distilled from DeepSeek-R1 based on Qwen and Llama.

page_1_chunk_0

# Contents

*   1 Introduction . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
    *   1.1 Contributions . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
    *   1.2 Summary of Evaluation Results . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4

*   2 Approach . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
    *   2.1 Overview . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
    *   2.2 DeepSeek-R1-Zero: Reinforcement Learning on the Base Model . . . . . . . . . . . 5
        *   2.2.1 Reinforcement Learning Algorithm . . . . . . . . . . . . . . . . . . . . . . . . 5
        *   2.2.2 Reward Modeling . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
        *   2.2.3 Training Template . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
        *   2.2.4 Performance, Self-evolution Process and Aha Moment of DeepSeek-R1-Zero 6
    *   2.3 DeepSeek-R1: Reinforcement Learning with Cold Start . . . . . . . . . . . . . . . . 9
        *   2.3.1 Cold Start . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9
        *   2.3.2 Reasoning-oriented Reinforcement Learning . . . . . . . . . . . . . . . . . . 10
        *   2.3.3 Rejection Sampling and Supervised Fine-Tuning . . . . . . . . . . . . . . . 10
        *   2.3.4 Reinforcement Learning for all Scenarios . . . . . . . . . . . . . . . . . . . . 11
    *   2.4 Distillation: Empower Small Models with Reasoning Capability . . . . . . . . . . . 11

*   3 Experiment . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11
    *   3.1 DeepSeek-R1 Evaluation . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 13
    *   3.2 Distilled Model Evaluation . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 14

*   4 Discussion . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 14
    *   4.1 Distillation v.s. Reinforcement Learning . . . . . . . . . . . . . . . . . . . . . . . . . 14
    *   4.2 Unsuccessful Attempts . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 15

*   5 Conclusion, Limitations, and Future Work . . . . . . . . . . . . . . . . . . . . . . . . . 16

*   A Contributions and Acknowledgments . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 20

page_2_chunk_0

...

# Appendix
## A. Contributions and Acknowledgments

This section lists individuals who contributed to the document, categorized into core contributors and general contributors.

### Core Contributors

<table>
  <thead>
    <tr>
      <th>Name</th>
      <th>Name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Daya Guo</td>
      <td>Hui Li</td>
    </tr>
    <tr>
      <td>Dejian Yang</td>
      <td>Jianzhong Guo</td>
    </tr>
    <tr>
      <td>Haowei Zhang</td>
      <td>Jiashi Li</td>
    </tr>
    <tr>
      <td>Junxiao Song</td>
      <td>Jingchang Chen</td>
    </tr>
    <tr>
      <td>Ruoyu Zhang</td>
      <td>Jingyang Yuan</td>
    </tr>
    <tr>
      <td>Runxin Xu</td>
      <td>Jinhao Tu</td>
    </tr>
    <tr>
      <td>Qihao Zhu</td>
      <td>Junjie Qiu</td>
    </tr>
    <tr>
      <td>Shirong Ma</td>
      <td>Junlong Li</td>
    </tr>
    <tr>
      <td>Peiyi Wang</td>
      <td>J.L. Cai</td>
    </tr>
    <tr>
      <td>Xiao Bi</td>
      <td>Jiaqi Ni</td>
    </tr>
    <tr>
      <td>Xiaokang Zhang</td>
      <td>Jian Liang</td>
    </tr>
    <tr>
      <td>Xingkai Yu</td>
      <td>Jin Chen</td>
    </tr>
    <tr>
      <td>Yu Wu</td>
      <td>Kai Dong</td>
    </tr>
    <tr>
      <td>Z.F. Wu</td>
      <td>Kai Hu*</td>
    </tr>
    <tr>
      <td>Zhibin Gou</td>
      <td>Kaichao You</td>
    </tr>
    <tr>
      <td>Zhihong Shao</td>
      <td>Kaige Gao</td>
    </tr>
    <tr>
      <td>Zhuoshu Li</td>
      <td>Kang Guan</td>
    </tr>
    <tr>
      <td>Ziyi Gao</td>
      <td>Kexin Huang</td>
    </tr>
    <tr>
      <td></td>
      <td>Kuai Yu</td>
    </tr>
  </tbody>
</table>

This table lists the "Core Contributors" to the document. It details 18 individuals who are primarily responsible for the work, alongside 19 additional individuals who are also considered core to the project. The asterisk (*) next to some names, like Kai Hu*, may indicate a special role or affiliation.

### Contributors

<table>
  <thead>
    <tr>
      <th>Name</th>
      <th>Name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Aixin Liu</td>
      <td>Lean Wang</td>
    </tr>
    <tr>
      <td>Bing Xue</td>
      <td>Lecong Zhang</td>
    </tr>
    <tr>
      <td>Bingxuan Wang</td>
      <td>Liang Zhao</td>
    </tr>
    <tr>
      <td>Bochao Wu</td>
      <td>Litong Wang</td>
    </tr>
    <tr>
      <td>Bei Feng</td>
      <td>Liyu Zhang</td>
    </tr>
    <tr>
      <td>Chengda Lu</td>
      <td>Lei Xu</td>
    </tr>
    <tr>
      <td>Chenggang Zhao</td>
      <td>Leyl Xia</td>
    </tr>
    <tr>
      <td>Chengqi Deng</td>
      <td>Mingchuan Zhang</td>
    </tr>
    <tr>
      <td>Chong Ruan</td>
      <td>Minghua Zhang</td>
    </tr>
    <tr>
      <td>Damai Dai</td>
      <td>Minghui Tang</td>
    </tr>
    <tr>
      <td>Deli Chen</td>
      <td>Mingxu Zhou</td>
    </tr>
    <tr>
      <td>Dongjie Ji</td>
      <td>Meng Li</td>
    </tr>
    <tr>
      <td>Erhang Li</td>
      <td>Miaojun Wang</td>
    </tr>
    <tr>
      <td>Fangyun Lin</td>
      <td>Mingming Li</td>
    </tr>
    <tr>
      <td>Fucong Dai</td>
      <td>Ning Tian</td>
    </tr>
    <tr>
      <td>Fuli Luo*</td>
      <td>Panpan Huang</td>
    </tr>
    <tr>
      <td>Guangbo Hao</td>
      <td>Peng Zhang</td>
    </tr>
    <tr>
      <td>Guanting Chen</td>
      <td>Qiancheng Wang</td>
    </tr>
    <tr>
      <td>Guowei Li</td>
      <td>Qinyu Chen</td>
    </tr>
    <tr>
      <td>H. Zhang</td>
      <td>Qiushi Du</td>
    </tr>
    <tr>
      <td>Hanwei Xu</td>
      <td>Ruiqi Ge*</td>
    </tr>
    <tr>
      <td>Honghui Ding</td>
      <td>Ruisong Zhang</td>
    </tr>
    <tr>
      <td>Huazuo Gao</td>
      <td>Ruizhe Pan</td>
    </tr>
    <tr>
      <td>Hui Qu</td>
      <td>Runji Wang</td>
    </tr>
    <tr>
      <td></td>
      <td>R.J. Chen</td>
    </tr>
    <tr>
      <td></td>
      <td>R.L. Jin</td>
    </tr>
  </tbody>
</table>

This table provides a comprehensive list of "Contributors" to the document, featuring 25 individuals in the left column and 25 in the right, indicating a broader group involved in the project. Similar to the core contributors, certain names like Fuli Luo* and Ruiqi Ge* are marked with an asterisk, potentially denoting specific roles or affiliations within this larger group.

page_20_chunk_0

<table>
  <thead>
    <tr>
      <th>Names (Column 1)</th>
      <th>Names (Column 2)</th>
    </tr>
  </thead>
  <tbody>
    <tr><td>Ruyi Chen</td><td>Y.X. Wei</td></tr>
    <tr><td>Shanghao Lu</td><td>Yang Zhang</td></tr>
    <tr><td>Shangyan Zhou</td><td>Yanhong Xu</td></tr>
    <tr><td>Shanhuang Chen</td><td>Yao Li</td></tr>
    <tr><td>Shengfeng Ye</td><td>Yao Zhao</td></tr>
    <tr><td>Shiyu Wang</td><td>Yaofeng Sun</td></tr>
    <tr><td>Shuiping Yu</td><td>Yaohui Wang</td></tr>
    <tr><td>Shunfeng Zhou</td><td>Yi Yu</td></tr>
    <tr><td>Shuting Pan</td><td>Yichao Zhang</td></tr>
    <tr><td>S.S. Li</td><td>Yifan Shi</td></tr>
    <tr><td>Shuang Zhou</td><td>Yiliang Xiong</td></tr>
    <tr><td>Shaoqing Wu</td><td>Ying He</td></tr>
    <tr><td>Shengfeng Ye</td><td>Yishi Piao</td></tr>
    <tr><td>Tao Yun</td><td>Yisong Wang</td></tr>
    <tr><td>Tian Pei</td><td>Yixuan Tan</td></tr>
    <tr><td>Tianyu Sun</td><td>Yiyang Ma*</td></tr>
    <tr><td>T. Wang</td><td>Yiyuan Liu</td></tr>
    <tr><td>Wangding Zeng</td><td>Yongqiang Guo</td></tr>
    <tr><td>Wen Liu</td><td>Yuan Ou</td></tr>
    <tr><td>Wenfeng Liang</td><td>Yuduan Wang</td></tr>
    <tr><td>Wenjun Gao</td><td>Yue Gong</td></tr>
    <tr><td>Wenqin Yu*</td><td>Yuheng Zou</td></tr>
    <tr><td>Wentao Zhang</td><td>Yujia He</td></tr>
    <tr><td>W.L. Xiao</td><td>Yunfan Xiong</td></tr>
    <tr><td>Wei An</td><td>Yuxiang Luo</td></tr>
    <tr><td>Xiaodong Liu</td><td>Yuxiang You</td></tr>
    <tr><td>Xiaohan Wang</td><td>Yuxuan Liu</td></tr>
    <tr><td>Xiaokang Chen</td><td>Yuyang Zhou</td></tr>
    <tr><td>Xiaotao Nie</td><td>Y.X. Zhu</td></tr>
    <tr><td>Xin Cheng</td><td>Yanping Huang</td></tr>
    <tr><td>Xin Liu</td><td>Yaohui Li</td></tr>
    <tr><td>Xin Xie</td><td>Yi Zheng</td></tr>
    <tr><td>Xingchao Liu</td><td>Yuchen Zhu</td></tr>
    <tr><td>Xinyu Yang</td><td>Yunxian Ma</td></tr>
    <tr><td>Xinyuan Li</td><td>Ying Tang</td></tr>
    <tr><td>Xuecheng Su</td><td>Yukun Zha</td></tr>
    <tr><td>Xuheng Lin</td><td>Yuting Yan</td></tr>
    <tr><td>X.Q. Li</td><td>Z.Z. Ren</td></tr>
    <tr><td>Xiangyue Jin</td><td>Zehui Ren</td></tr>
    <tr><td>Xiaojin Shen</td><td>Zhangli Sha</td></tr>
    <tr><td>Xiaosha Chen</td><td>Zhe Fu</td></tr>
    <tr><td>Xiaowen Sun</td><td>Zhean Xu</td></tr>
    <tr><td>Xiaoxiang Wang</td><td>Zhenda Xie</td></tr>
    <tr><td>Xinnan Song</td><td>Zhengyan Zhang</td></tr>
    <tr><td>Xinyi Zhou</td><td>Zhewen Hao</td></tr>
    <tr><td>Xianzu Wang</td><td>Zhicheng Ma</td></tr>
    <tr><td>Xinxia Shan</td><td>Zhigang Yan</td></tr>
    <tr><td>Y.K. Li</td><td>Zhiyu Wu</td></tr>
    <tr><td>Y.Q. Wang</td><td>Zihui Gu</td></tr>
  </tbody>
</table>
This table presents a comprehensive list of 98 names, organized into two columns with 49 names each. These names likely represent individuals such as authors or contributors to a document. Notably, two names, 'Wenqin Yu*' and 'Yiyang Ma*', are marked with an asterisk, which may indicate a specific affiliation or special designation.

21

page_21_chunk_0

<table>
  <thead>
    <tr>
      <th>Author List 1</th>
      <th>Author List 2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Zijia Zhu</td>
      <td>Zhen Huang</td>
    </tr>
    <tr>
      <td>Zijun Liu*</td>
      <td>Zhipeng Xu</td>
    </tr>
    <tr>
      <td>Zilin Li</td>
      <td>Zhongyu Zhang</td>
    </tr>
    <tr>
      <td>Ziwei Xie</td>
      <td>Zhen Zhang</td>
    </tr>
    <tr>
      <td>Ziyang Song</td>
      <td></td>
    </tr>
    <tr>
      <td>Zizheng Pan</td>
      <td></td>
    </tr>
  </tbody>
</table>

This table presents a list of authors, organized into two distinct columns. A total of ten authors are listed across both columns. Notably, Zijun Liu is marked with an asterisk, indicating their departure from the team.

Within each role, authors are listed alphabetically by the first name. Names marked with * denote individuals who have departed from our team.

Page 22

Now, let’s set up the vector database ChromaDB:

In [15]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="documents")

print(f"ChromeDB contains {collection.count()} documents.")

ChromeDB contains 89 documents.


For the embedding model we use the Gemini API along with the task type for Retrieval — this will improve the quality of the embeddings. But, you can use any embedding model here like a BERT-based model.

For each chunk create the embeddings with an embedding model and save them along with the documents in the vector database:

In [16]:
chunk_texts = [ch["text"] for ch in all_chunks_again]

embedding_response = client.models.embed_content(
    model = "text-embedding-004",
    contents = chunk_texts,
    config = types.EmbedContentConfig(task_type = "RETRIEVAL_DOCUMENT")
)

# embedding_response is a dydantic model of type EmbedContentResponseDict:
actual_embeddings = [e.values for e in embedding_response.embeddings]

# Update documents along with the enbeddings to ChromaDB
collection.upsert(
    ids = [str(i) for i in range(len(chunk_texts))],
    embeddings = actual_embeddings,
    metadatas = [{"text": doc} for doc in chunk_texts]
)

In [17]:
print(f"ChromeDB contains {collection.count()} documents.")

ChromeDB contains 90 documents.


Now, we can embed a user query, perform a similarity search with all embedded documents to find the best matching top k documents.

In [ ]:
def retrieve_relevant_documents(query, top_k=10):
    # generate embedding for the query
    query_embedding = client.models.embed_content(
        model = "text-embedding-004",
        contents = [query],
        config = types.EmbedContentConfig(task_type="RETRIEVAL_QUERY")
    )

    query_embedding_values = query_embedding.embeddings[0].values

    # Perform similarity search in ChromaDB
    results = collection.query(
        query_embeddings = [query_embedding_values],
        n_results = top_k
    )

    # Extract results
    retrieved_documents = [doc["text"] for doc in results["metadatas"][0]]

    return retrieved_documents    

Final RAG-part: Generate the answer

Finally, we can feed the top k retrieved documents as context along with the query to the LLM to generate the final answer:

In [19]:
query = "Why is DeepSeek considered a major breakthrough? How much better did DeepSeek-R1-Zero perform on the benchmarks compared to other models?"

relevant_docs = retrieve_relevant_documents(query)

print("\nTop relevant documents:")
for n, doc in enumerate(relevant_docs):
    print("\n\nDocument #", n)
    print("*", doc)

context = "\n\n".join(relevant_docs)

answer_response = f"""Use the following context to answer the question:
Context: 
{context}
Question: 
{query}
Answer:
"""

response = client.models.generate_content(
    model = "gemini-2.5-flash-preview-05-20",
    contents = answer_response,
)

print("\n=== Gemini 2.5 Flash Answer using relevant parts of the pdf document ===")
print(response.text)


Top relevant documents:


Document # 0
* DeepSeek-R1-Zero aims to attain robust reasoning capabilities without the need for any supervised fine-tuning data. This is a noteworthy achievement, as it underscores the model's ability to learn and generalize effectively through RL alone. Additionally, the performance of DeepSeek-R1-Zero can be further augmented through the application of majority voting. For example, when majority voting is employed on the AIME benchmark, DeepSeek-R1-Zero's performance escalates from 71.0% to 86.7%, thereby exceeding the performance of OpenAI-o1-0912. The ability of DeepSeek-R1-Zero to achieve such competitive performance, both with and without majority voting, highlights its strong foundational capabilities and its potential for further advancements in reasoning tasks.


Document # 1
* DeepSeek-R1 also delivers impressive results on IF-Eval, a benchmark designed to assess a model's ability to follow format instructions. These improvements can be linked to 

Displaying again with Markdown enabled.

In [25]:
print("\nTop relevant documents:")
for n, doc in enumerate(relevant_docs):
    print("\n\nDocument #", n)
    display(Markdown("* " + doc))

print("\n=== Gemini 2.5 Flash Answer using relevant parts of the pdf document ===")
display(Markdown(response.text))


Top relevant documents:


Document # 0


* DeepSeek-R1-Zero aims to attain robust reasoning capabilities without the need for any supervised fine-tuning data. This is a noteworthy achievement, as it underscores the model's ability to learn and generalize effectively through RL alone. Additionally, the performance of DeepSeek-R1-Zero can be further augmented through the application of majority voting. For example, when majority voting is employed on the AIME benchmark, DeepSeek-R1-Zero's performance escalates from 71.0% to 86.7%, thereby exceeding the performance of OpenAI-o1-0912. The ability of DeepSeek-R1-Zero to achieve such competitive performance, both with and without majority voting, highlights its strong foundational capabilities and its potential for further advancements in reasoning tasks.



Document # 1


* DeepSeek-R1 also delivers impressive results on IF-Eval, a benchmark designed to assess a model's ability to follow format instructions. These improvements can be linked to the inclusion of instruction-following data during the final stages of supervised fine-tuning (SFT) and RL training. Furthermore, remarkable performance is observed on AlpacaEval2.0 and ArenaHard, indicating DeepSeek-R1's strengths in writing tasks and open-domain question answering. Its significant outperformance of DeepSeek-V3 underscores the generalization benefits of large-scale RL, which not only boosts reasoning capabilities but also improves performance across diverse domains. Moreover, the summary lengths generated by DeepSeek-R1 are concise, with an average of 689 tokens on ArenaHard and 2,218 characters on AlpacaEval 2.0. This indicates that



Document # 2


...

* Table 2 | Comparison of DeepSeek-R1-Zero and OpenAI o1 models on reasoning-related benchmarks.

<table border="1">
<thead>
<tr>
<th>Model</th>
<th colspan="2">AIME 2024</th>
<th>MATH-500</th>
<th>GPQA Diamond</th>
<th>LiveCode Bench</th>
<th>CodeForces</th>
</tr>
<tr>
<th></th>
<th>pass@1</th>
<th>cons@64</th>
<th>pass@1</th>
<th>pass@1</th>
<th>pass@1</th>
<th>rating</th>
</tr>
</thead>
<tbody>
<tr>
<td>OpenAI-o1-mini</td>
<td>63.6</td>
<td>80.0</td>
<td>90.0</td>
<td>60.0</td>
<td>53.8</td>
<td>1820</td>
</tr>
<tr>
<td>OpenAI-o1-0912</td>
<td>74.4</td>
<td>83.3</td>
<td>94.8</td>
<td>77.3</td>
<td>63.4</td>
<td>1843</td>
</tr>
<tr>
<td>DeepSeek-R1-Zero</td>
<td>71.0</td>
<td>86.7</td>
<td>95.9</td>
<td>73.3</td>
<td>50.0</td>
<td>1444</td>
</tr>
</tbody>
</table>

Table 2 provides a comparison of DeepSeek-R1-Zero's performance against two OpenAI o1 models on various reasoning-related benchmarks. DeepSeek-R1-Zero achieved the highest pass@1 score on MATH-500 (95.9%) and the highest cons@64 score on AIME 2024 (86.7%), demonstrating superior performance in these specific areas compared to both OpenAI models. While excelling in some metrics, DeepSeek-R1-Zero had a lower rating on CodeForces (1444) and a lower pass@1 on LiveCode Bench (50.0%) than the OpenAI models.



Document # 9


* However, DeepSeek-R1-Zero encounters challenges such as poor readability, and language mixing. To address these issues and further enhance reasoning performance, we introduce DeepSeek-R1, which incorporates a small amount of cold-start data and a multi-stage training pipeline. Specifically, we begin by collecting thousands of cold-start data to fine-tune the DeepSeek-V3-Base model. Following this, we perform reasoning-oriented RL like DeepSeek-R1-Zero. Upon nearing convergence in the RL process, we create new SFT data through rejection sampling on the RL checkpoint, combined with supervised data from DeepSeek-V3 in domains such as writing, factual QA, and self-cognition, and then retrain the DeepSeek-V3-Base model. After fine-tuning with the new data, the checkpoint undergoes an additional RL process, taking into account prompts from all scenarios. After these steps, we obtained a checkpoint referred to as DeepSeek-R1, which achieves performance on par with OpenAI-o1-1217.


=== Gemini 2.5 Flash Answer using relevant parts of the pdf document ===


DeepSeek is considered a major breakthrough primarily because **DeepSeek-R1-Zero demonstrates robust reasoning capabilities achieved solely through Reinforcement Learning (RL), without the need for any supervised fine-tuning (SFT) data**. This highlights its ability to learn and generalize effectively through RL alone, showcasing an intrinsic development within the model to solve complex reasoning tasks.

Furthermore, DeepSeek-R1 (the more advanced version building upon R1-Zero's foundations) delivers impressive generalized performance:
*   It achieves performance on par with or even slightly surpasses top models like OpenAI-o1-1217 on various reasoning tasks (e.g., AIME 2024 and MATH-500).
*   It shows remarkable performance on instruction-following (IF-Eval), writing tasks (AlpacaEval2.0, ArenaHard), open-domain question answering, and long-context QA (FRAMES).
*   It significantly outperforms DeepSeek-V3 across diverse domains and knowledge benchmarks (MMLU, MMLU-Pro, GPQA Diamond, SimpleQA), showcasing the generalization benefits of large-scale RL.
*   It demonstrates expert-level performance in coding competition tasks, achieving a 2,029 Elo rating on Codeforces, outperforming 96.3% of human participants.

Regarding **DeepSeek-R1-Zero's performance compared to other models on the benchmarks (specifically OpenAI-o1-0912 and OpenAI-o1-mini),** based on Table 2 and accompanying text:

*   **AIME 2024:**
    *   **pass@1:** DeepSeek-R1-Zero achieved **71.0%**, which was lower than OpenAI-o1-0912 (74.4%) but higher than OpenAI-o1-mini (63.6%).
    *   **cons@64:** DeepSeek-R1-Zero achieved the highest score at **86.7%**, surpassing both OpenAI-o1-0912 (83.3%) and OpenAI-o1-mini (80.0%). The text also notes that this 86.7% was achieved through majority voting and exceeded OpenAI-o1-0912's performance.
*   **MATH-500 (pass@1):** DeepSeek-R1-Zero achieved the highest score at **95.9%**, outperforming OpenAI-o1-0912 (94.8%) and OpenAI-o1-mini (90.0%).
*   **GPQA Diamond (pass@1):** DeepSeek-R1-Zero scored **73.3%**, which was higher than OpenAI-o1-mini (60.0%) but lower than OpenAI-o1-0912 (77.3%).
*   **LiveCode Bench (pass@1):** DeepSeek-R1-Zero scored **50.0%**, which was lower than both OpenAI-o1-0912 (63.4%) and OpenAI-o1-mini (53.8%).
*   **CodeForces (rating):** DeepSeek-R1-Zero had a rating of **1444**, which was lower than both OpenAI-o1-0912 (1843) and OpenAI-o1-mini (1820).

In summary, DeepSeek-R1-Zero showed superior performance in some key areas like AIME 2024 (cons@64) and MATH-500, while demonstrating competitive but sometimes lower performance in other benchmarks like LiveCode Bench and CodeForces, compared to its OpenAI o1 counterparts. Its initial AIME 2024 pass@1 score improved significantly from 15.6% to 71.0% during its RL training process, making it comparable to OpenAI’s 01-0912.

In [ ]:
# import sys
# print(sys.executable)

c:\Program Files\Python311\python.exe
